# Solving models with rational and floating point.

Using [cobrapy](http://opencobra.github.io/cobrapy/) (version 0.3.2 or later), we have interfaces to serveral floating point solvers ([gurobi](http://www.gurobi.com), [MOSEK](http://www.mosek.com/), [CPLEX](http://www-01.ibm.com/software/commerce/optimization/cplex-optimizer/), and two different versions of [GLPK](https://www.gnu.org/software/glpk/)). Additionally, we can use the rational solving capabilities of GLPK and esolver (also known as [QSopt_ex](http://www.dii.uchile.cl/~daespino/ESolver_doc/main.html)). We will use all of these solvers on all the models in the collection.

In [1]:
from numpy import array
import pandas
import sympy
import scipy.io

import cobra

pandas.set_option("display.max_rows", 100)

We will verify all of our solutions by writing out the Stoichiometrix matrix $\mathbf S$ using the [sympy](http://sympy.org/) symbolic math library. The total error will be equivalent to 
$$ \sum \left|\mathbf S \cdot v \right| $$

In [2]:
def convert_to_rational(value):
    return sympy.Rational("%.15g" % value)

def construct_exact_S(model):
    # intialize to 0
    S = sympy.SparseMatrix(len(model.metabolites), len(model.reactions), 0)
    # populate with stoichiometry
    for i, r in enumerate(model.reactions):
        for met, value in r._metabolites.iteritems():
            S[model.metabolites.index(met), i] = convert_to_rational(value)
    return S

def total_error(S, v):
    return sum(abs(i) for i in S * v)

Some of these models were exported from [BiGG](http://bigg.ucsd.edu/bigg/export.pl) as SBML, and others were downloaded from their respective publications. All of the models are available in a [git repository](https://github.com/opencobra/m_model_collection). They were parsed into a single mat file by an [included script](load_models.ipynb).

In [3]:
models = []
model_file = scipy.io.loadmat("all_models.mat")
for possible_name in sorted(model_file.keys()):
    if possible_name.startswith("_"):
        continue
    model = cobra.io.mat.from_mat_struct(model_file[possible_name], model_id=possible_name)
    models.append(model)

These models all compute rationally with esolver and also with the other floating point solvers, as shown below. They also compute with floating point solvers in the [COBRA toolbox](http://dx.doi.org/doi:10.1038/nprot.2011.308), as shown [here](http://nbviewer.ipython.org/github/opencobra/m_model_collection/blob/master/MATLAB.ipynb).

In [4]:
results = {}
exact_results = {}
errors = {}
for m in models:
    S = construct_exact_S(m)
    S_float = m.to_array_based_model().S
    rational_solution = m.optimize(solver="esolver", rational_solution=True)
    rational_v = sympy.Matrix(rational_solution.x)
    exact_results[m.id] = {"Rational": rational_solution.f,
                           "Decimal": float(rational_solution.f),
                           "Error": total_error(S, rational_v),
                           # Ensure the upper and lwoer bounds are satisfied.
                           "Bounds": all([r.upper_bound >= value >= r.lower_bound
                                              for r, value in zip(m.reactions, rational_v)])}
    # solve this model with all the solvers
    solutions = {solver: m.optimize(solver=solver)
                 for solver in cobra.solvers.solver_dict}
    solutions["cglpk_exact"] = m.optimize(solver="cglpk", exact=True)
    # store the objective value and errors
    results[m.id] = {k: v.f for k, v in solutions.iteritems()}
    errors[m.id] = {k: sum(abs(S_float * array(v.x))) for k, v in solutions.iteritems()}
# format the results as pandas dataframes
exact_results = pandas.DataFrame.from_dict(exact_results).T
results = pandas.DataFrame.from_dict(results)
errors = pandas.DataFrame.from_dict(errors)

For all of these models, we can demonstrate they satisfy the model constraints using exact operations.

$$\sum\left|\mathbf S \cdot v\right| = 0$$

In [5]:
abs(exact_results.Error).max()

0

$$ub \ge v \ge lb$$

In [6]:
exact_results.Bounds.all()

True

Here are objective values of the rational results provided by esolver:

In [7]:
exact_results[["Decimal", "Rational"]]

,Decimal,Rational
AORYZAE_COBRA,25.6824,10504392200845731410/409011287584573141
AbyMBEL891,119.233,386664062500000/3242927908653
AraGEM,10,10
GSMN_TB,0.1312623,576559343750000/4392422515431629
PpaMBEL1254,78.70059,4725000000000/60037666259
PpuMBEL1071,132.3654,644062500000/4865793703
STM_v1_0,0.4778337,178576000/373720009
S_coilicolor_fixed,860.0888,408741530000000000/475231759036371
SpoMBEL1693,63.78566,1237500000000/19400913101
T_Maritima,0.359469,180000000/500738623


Here is the $\sum\left|\mathbf S \cdot v\right|$ error computed using floating point operations for every solver. When computed rationally with esolver above, this value was exactly 0. Howver, when rounding the fractional values to floating point, there is a very small amount of resulting error, so even esolver does not give 0 error for this computation.

In [8]:
errors.T

,cglpk,cglpk_exact,cplex,esolver,glpk,gurobi,mosek
AORYZAE_COBRA,8.397096e-12,9.542499e-12,7.880516e-12,7.278331e-12,9.208338e-12,1.658178e-11,1.755370e-11
AbyMBEL891,1.030879e-11,6.538390e-12,1.652062e-11,6.093565e-12,1.168024e-11,2.098006e-11,2.173014e-11
AraGEM,4.518242e-12,3.761263e-12,1.031621e-11,3.953687e-12,5.721206e-12,1.492412e-11,1.607253e-11
GSMN_TB,3.265029e-10,1.438631e-10,1.465796e-10,1.562416e-10,1.580843e-07,6.897009e-09,8.171375e-10
PpaMBEL1254,5.524246e-12,5.503863e-12,1.653475e-11,1.599098e-12,5.808043e-12,6.458234e-12,1.327733e-11
PpuMBEL1071,9.213515e-12,1.006636e-07,6.212253e-12,5.611067e-12,6.928756e-12,1.996325e-11,1.482070e-11
STM_v1_0,2.442772e-12,7.615986e-09,6.747805e-13,4.133462e-13,4.027310e-12,2.884140e-11,1.424397e-11
S_coilicolor_fixed,6.094508e-11,3.403941e-11,5.070666e-11,3.497688e-11,4.686266e-11,5.399156e-11,2.215057e-10
SpoMBEL1693,9.424184e-12,6.328653e-12,7.634511e-12,5.512237e-12,8.155074e-12,9.632115e-12,2.056227e-11
T_Maritima,1.149696e-12,1.017970e-09,1.312999e-12,1.125577e-12,1.578348e-12,4.792797e-12,1.510734e-11


Here are all the computed biomass flux rates for each solver and model.

In [9]:
results.T

,cglpk,cglpk_exact,cplex,esolver,glpk,gurobi,mosek
AORYZAE_COBRA,25.682402,25.682402,25.682402,25.682402,25.682402,25.682402,25.682402
AbyMBEL891,119.233012,119.233012,119.233012,119.233012,119.233012,119.233012,119.233012
AraGEM,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
GSMN_TB,0.131262,0.131262,0.131262,0.131262,0.131262,0.131262,0.131262
PpaMBEL1254,78.700594,78.700594,78.700594,78.700594,78.700594,78.700594,78.700594
PpuMBEL1071,132.365353,132.365353,132.365353,132.365353,132.365353,132.365353,132.365353
STM_v1_0,0.477834,0.477834,0.477834,0.477834,0.477834,0.477834,0.477834
S_coilicolor_fixed,860.088835,860.088835,860.088835,860.088835,860.088835,860.088835,860.088835
SpoMBEL1693,63.785658,63.785658,63.785658,63.785658,63.785658,63.785658,63.785658
T_Maritima,0.359469,0.359469,0.359469,0.359469,0.359469,0.359469,0.359469


The objectives computed for these solvers are effectively the same as those computed by esolver.

In [10]:
differences = (results - results.ix["esolver"]).T
differences.pop("esolver")
differences

,cglpk,cglpk_exact,cplex,glpk,gurobi,mosek
AORYZAE_COBRA,5.329071e-14,3.552714e-15,3.552714e-15,-3.552714e-15,0.000000e+00,-9.947598e-14
AbyMBEL891,1.136868e-13,0.000000e+00,-5.684342e-14,1.136868e-13,-2.415845e-13,-7.105427e-14
AraGEM,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
GSMN_TB,-4.656553e-13,0.000000e+00,3.608225e-16,1.011393e-08,9.173218e-14,4.979073e-12
PpaMBEL1254,0.000000e+00,0.000000e+00,-1.847411e-13,0.000000e+00,0.000000e+00,-7.105427e-14
PpuMBEL1071,8.526513e-14,5.117897e-10,5.684342e-14,8.526513e-14,5.684342e-14,-1.136868e-13
STM_v1_0,3.863576e-14,4.957084e-10,-1.498801e-15,1.086908e-13,-4.996004e-16,4.814482e-13
S_coilicolor_fixed,1.136868e-13,0.000000e+00,0.000000e+00,1.136868e-13,0.000000e+00,-1.250555e-12
SpoMBEL1693,-1.421085e-14,0.000000e+00,-1.421085e-14,7.105427e-14,-2.842171e-14,-6.039613e-13
T_Maritima,-3.214096e-14,9.456913e-11,3.330669e-16,-2.114975e-14,-3.330669e-16,-3.308465e-14
